In this notebook, plated reader data from the long-duration experiments described in Figure 2, Figure 3F, Supplementary Figures 13-15 and Supplementary Figure 28 will be analyzed and plots depicted in these figures generated. These plots were then formatted into the full figures in Affinity Designer.

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize
import murraylab_tools.biotek as btek
import itertools
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from cycler import cycler

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('legend', frameon=False)
mpl.rc('figure', figsize=(8.5,15))
%matplotlib qt

In [2]:
from platform import python_version
print(python_version())

3.7.12


Below will load data and generate tidy dataframes. Alternatively can load data frames.

In [3]:
# load data for first 384 well plate and associated metadata
plate1_p1 = pd.read_csv('./20211025_1_plate1.csv')
plate1_p2 = pd.read_csv('./20211025_1_plate2.csv')
plate1_p3 = pd.read_csv('./20211025_1_plate3.csv')
plate1_p4 = pd.read_csv('./20211025_1_plate4.csv')
plate1_p5 = pd.read_csv('./20211025_1_plate5.csv')
plate1_p6 = pd.read_csv('./20211025_1_plate6.csv')
plate1_p7 = pd.read_csv('./20211025_1_plate7.csv')
plate1_p8 = pd.read_csv('./20211025_1_plate8.csv')
plate1_p9 = pd.read_csv('./20211025_1_plate9.csv')
plate1_p10 = pd.read_csv('./20211025_1_plate10.csv')
plate1_p11 = pd.read_csv('./20211025_1_plate11.csv')
plate1_p12 = pd.read_csv('./20211025_1_plate12.csv')
plate1_p13 = pd.read_csv('./20211025_1_plate13.csv')
plate1_p14 = pd.read_csv('./20211025_1_plate14.csv')
plate1_p15 = pd.read_csv('./20211025_1_plate15.csv')
plate1_p16 = pd.read_csv('./20211025_1_plate16.csv')
plate1_dict={'p1':plate1_p1,'p2':plate1_p2,'p3':plate1_p3,'p4':plate1_p4,'p5':plate1_p5,
             'p6':plate1_p6,'p7':plate1_p7,'p8':plate1_p8,'p9':plate1_p9,'p10':plate1_p10,
             'p11':plate1_p11,'p12':plate1_p12,'p13':plate1_p13,'p14':plate1_p14,'p15':plate1_p15,
             'p16':plate1_p16}
plate1_metadata = pd.read_csv('./20211025_metadata_plate1.csv')

columns = list(plate1_metadata.columns)
columns += ['plate','channel','read','measurement']
cols = np.arange(1,25)
rows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']
reads = plate1_p1['read'].unique()

# add metadata
data = []
for i in np.arange(1,17):
    df = plate1_dict['p'+str(i)]
    for row in rows:
        for col in cols:
            for read in reads:
                value = df.loc[(df.row==row)&(df.read==read),str(int(col))].values[0]
                channel = read.split('_')[0]
                read_num = read.split('_')[1][0]
                data_add = list(plate1_metadata.loc[plate1_metadata.well==row+str(int(col)),:].values.flatten())
                data_add+=[i,channel,read_num,value]
                data.append(data_add)
df_master_plate1 = pd.DataFrame(columns=columns,data=data)

# get averages
df_master_plate1_avg = df_master_plate1.groupby(['well','strain','sal','las','iptg','chlor','cole1','rep','plate','channel'],as_index=False)['measurement'].mean()
df_master_plate1_avg['plate'] = df_master_plate1_avg['plate'].astype(int)
df_master_plate1_avg['rep'] = df_master_plate1_avg['rep'].astype(int)


In [4]:
# do the same for the second 384 well plate
plate2_p1 = pd.read_csv('./20211025_2_plate1.csv')
plate2_p2 = pd.read_csv('./20211025_2_plate2.csv')
plate2_p3 = pd.read_csv('./20211025_2_plate3.csv')
plate2_p4 = pd.read_csv('./20211025_2_plate4.csv')
plate2_p5 = pd.read_csv('./20211025_2_plate5.csv')
plate2_p6 = pd.read_csv('./20211025_2_plate6.csv')
plate2_p7 = pd.read_csv('./20211025_2_plate7.csv')
plate2_p8 = pd.read_csv('./20211025_2_plate8.csv')
plate2_p9 = pd.read_csv('./20211025_2_plate9.csv')
plate2_p10 = pd.read_csv('./20211025_2_plate10.csv')
plate2_p11 = pd.read_csv('./20211025_2_plate11.csv')
plate2_p12 = pd.read_csv('./20211025_2_plate12.csv')
plate2_p13 = pd.read_csv('./20211025_2_plate13.csv')
plate2_p14 = pd.read_csv('./20211025_2_plate14.csv')
plate2_p15 = pd.read_csv('./20211025_2_plate15.csv')
plate2_p16 = pd.read_csv('./20211025_2_plate16.csv')
plate2_dict={'p1':plate2_p1,'p2':plate2_p2,'p3':plate2_p3,'p4':plate2_p4,'p5':plate2_p5,
             'p6':plate2_p6,'p7':plate2_p7,'p8':plate2_p8,'p9':plate2_p9,'p10':plate2_p10,
             'p11':plate2_p11,'p12':plate2_p12,'p13':plate2_p13,'p14':plate2_p14,'p15':plate2_p15,
             'p16':plate2_p16}
plate2_metadata = pd.read_csv('./20211025_metadata_plate2.csv')

columns = list(plate2_metadata.columns)
columns += ['plate','channel','read','measurement']
cols = np.arange(1,25,2)
rows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']
reads = plate2_p1['read'].unique()
data = []
for i in np.arange(1,17):
    df = plate2_dict['p'+str(i)]
    for row in rows:
        for col in cols:
            for read in reads:
                value = df.loc[(df.row==row)&(df.read==read),str(int(col))].values[0]
                channel = read.split('_')[0]
                read_num = read.split('_')[1][0]
                data_add = list(plate2_metadata.loc[plate2_metadata.well==row+str(int(col)),:].values.flatten())
                data_add+=[i,channel,read_num,value]
                data.append(data_add)
df_master_plate2 = pd.DataFrame(columns=columns,data=data)
df_master_plate2_avg = df_master_plate2.groupby(['well','strain','sal','las','iptg','chlor','cole1','rep','plate','channel'],as_index=False)['measurement'].mean()
df_master_plate2_avg['plate'] = df_master_plate2_avg['plate'].astype(int)
df_master_plate2_avg['rep'] = df_master_plate2_avg['rep'].astype(int)


In [5]:
# make GFP cumsum dataframe for plate 1
df_gfp_plate1 = df_master_plate1_avg.loc[df_master_plate1_avg.channel=='sfGFP',:]
plate1_gfp_array_cumsum = np.array(df_gfp_plate1.loc[:,:])
wells = df_gfp_plate1.well.unique()

for well in wells:
    for gen in np.arange(2,17):
        plate1_gfp_array_cumsum[(plate1_gfp_array_cumsum[:,0]==well)&(plate1_gfp_array_cumsum[:,8]==gen),10] = \
        plate1_gfp_array_cumsum[(plate1_gfp_array_cumsum[:,0]==well)&(plate1_gfp_array_cumsum[:,8]==gen-1),10] +\
        plate1_gfp_array_cumsum[(plate1_gfp_array_cumsum[:,0]==well)&(plate1_gfp_array_cumsum[:,8]==gen),10]
df_gfp_plate1_cumsum = pd.DataFrame(columns=df_gfp_plate1.columns,data=plate1_gfp_array_cumsum)

In [6]:
# make GFP cumsum dataframe for plate 2
df_gfp_plate2 = df_master_plate2_avg.loc[df_master_plate2_avg.channel=='sfGFP',:]
plate2_gfp_array_cumsum = np.array(df_gfp_plate2.loc[:,:])
wells = df_gfp_plate2.well.unique()

for well in wells:
    for gen in np.arange(2,17):
        plate2_gfp_array_cumsum[(plate2_gfp_array_cumsum[:,0]==well)&(plate2_gfp_array_cumsum[:,8]==gen),10] = \
        plate2_gfp_array_cumsum[(plate2_gfp_array_cumsum[:,0]==well)&(plate2_gfp_array_cumsum[:,8]==gen-1),10] +\
        plate2_gfp_array_cumsum[(plate2_gfp_array_cumsum[:,0]==well)&(plate2_gfp_array_cumsum[:,8]==gen),10]
df_gfp_plate2_cumsum = pd.DataFrame(columns=df_gfp_plate2.columns,data=plate2_gfp_array_cumsum)

In [7]:
df_gfp_plate1_cumsum.head()


,well,strain,sal,las,iptg,chlor,cole1,rep,plate,channel,measurement
0,A1,1x diff 2x bxb1,10,0.01,10,-,carb,1,1,sfGFP,1897
1,A1,1x diff 2x bxb1,10,0.01,10,-,carb,1,2,sfGFP,6959.5
2,A1,1x diff 2x bxb1,10,0.01,10,-,carb,1,3,sfGFP,12944.5
3,A1,1x diff 2x bxb1,10,0.01,10,-,carb,1,4,sfGFP,18703
4,A1,1x diff 2x bxb1,10,0.01,10,-,carb,1,5,sfGFP,23136.5


In [8]:
# combine all data and make dataframes with averages
df_master_avg = pd.concat([df_master_plate1_avg,df_master_plate2_avg])
df_master_avg['plate'] = df_master_avg['plate'].astype(int)
df_master_avg['rep'] = df_master_avg['rep'].astype(int)
df_gfp_cumsum = pd.concat([df_gfp_plate1_cumsum,df_gfp_plate2_cumsum])
df_od = df_master_avg.loc[df_master_avg.channel=='OD700',:]
df_rfp = df_master_avg.loc[df_master_avg.channel=='mScarlet']
df_gfp = df_master_avg.loc[df_master_avg.channel=='sfGFP',:]
df_gfp_cumsum['measurement'] = df_gfp_cumsum['measurement'].astype('float')
df_gfp['measurement'] = df_gfp['measurement'].astype('float')
df_od['measurement'] = df_od['measurement'].astype('float')
df_rfp['measurement'] = df_rfp['measurement'].astype('float')
df_gfp_cumsum_avg = df_gfp_cumsum.groupby(['strain','sal','las','iptg','chlor','cole1','plate'],as_index=False)['measurement'].mean()
df_gfp_avg = df_gfp.groupby(['strain','sal','las','iptg','chlor','cole1','plate'],as_index=False)['measurement'].mean()
df_od_avg = df_od.groupby(['strain','sal','las','iptg','chlor','cole1','plate'],as_index=False)['measurement'].mean()
df_gfp_cumsum_avg['std'] = df_gfp_cumsum.groupby(['strain','sal','las','iptg','chlor','cole1','plate'])['measurement'].agg(np.std).values
df_gfp_avg['std'] = df_gfp.groupby(['strain','sal','las','iptg','chlor','cole1','plate'])['measurement'].agg(np.std).values
df_od_avg['std'] = df_od.groupby(['strain','sal','las','iptg','chlor','cole1','plate'])['measurement'].agg(np.std).values
df_rfp_avg = df_rfp.groupby(['strain','sal','las','iptg','chlor','cole1','plate'],as_index=False)['measurement'].mean()
df_rfp_avg['std'] = df_rfp.groupby(['strain','sal','las','iptg','chlor','cole1','plate'])['measurement'].agg(np.std).values

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [9]:
# save csvs for easier access
df_gfp.to_csv('20211025_gfp.csv')
df_rfp.to_csv('20211025_rfp.csv')
df_od.to_csv('20211025_OD700.csv')
df_gfp_cumsum.to_csv('20211025_gfp_cumsum.csv')
df_gfp_cumsum_avg.to_csv('20211025_gfp_cumsum_avg.csv')
df_gfp_avg.to_csv('20211025_gfp_avg.csv')
df_rfp_avg.to_csv('20211025_rfp_avg.csv')
df_od_avg.to_csv('20211025_OD700_avg.csv')

Instead of running the above code, can instead load the tidy data files below.

In [28]:
df_gfp = pd.read_csv('20211025_gfp.csv')
df_gfp_cumsum = pd.read_csv('20211025_gfp_cumsum.csv')
df_gfp_cumsum_avg = pd.read_csv('20211025_gfp_cumsum_avg.csv')
df_gfp_avg = pd.read_csv('20211025_gfp_avg.csv')
df_rfp_avg = pd.read_csv('20211025_rfp_avg.csv')
df_od_avg = pd.read_csv('20211025_OD700_avg.csv')

Below will generate the time course plots of average GFP production per plate generation shown in Figure 2 A-C, and Figure S11 A-C. For clarity, individual replicates and error bars are not shown here, but all replicates are shown in panel D which shows total cumulative production after 16 plate generations.

In [33]:
#noerrorbar plot new colors, diverging plaette, same colors for +/- chlor
abx = df_gfp.chlor.unique()
las = 0.01
iptgs = [10,50]
sals = [10,15,20,30]
chlor_colors = ['#4575b4','#abd9e9', '#fdae61', '#d73027']
nochlor_colors = ['#4575b4','#abd9e9', '#fdae61', '#d73027']
naive_colors = [(150/255,150/255,150/255),(37/255,37/255,37/255)]
strains = np.array(['1x diff 2x bxb1','split pir 2x bxb1'])
legend_list = []
cole1='carb' # set to 'kan' to generate timecourse plots for Figure 2, 'carb' for timecourse plots in Figure S11

lw=1.5
marker_size=5
offsets=np.linspace(-0.3,0.3,8)*0
fig, ax = plt.subplots(2,3,sharey=True,sharex=True,figsize=(12,15))

for i, strain in enumerate(strains):
    for j, iptg in enumerate(iptgs):
        ax[j,i].set_title(f'{strain}: {iptg}$\mu$M IPTG',fontsize=14)
        for k, sal in enumerate(sals):
            ax[j,i].plot(df_gfp_avg.loc[(df_gfp_avg.strain==strain) &\
                          (df_gfp_avg.chlor=='-') &\
                          (df_gfp_avg.iptg==iptg) &\
                          (df_gfp_avg.sal==sal)&\
                      (df_gfp_avg.cole1==cole1),'plate'].values+offsets[2*k],
                            df_gfp_avg.loc[(df_gfp_avg.strain==strain) &\
                          (df_gfp_avg.chlor=='-') &\
                          (df_gfp_avg.iptg==iptg) &\
                          (df_gfp_avg.sal==sal)&\
                      (df_gfp_avg.cole1==cole1),'measurement'],
                             linestyle='-',linewidth=lw,color=nochlor_colors[k],alpha=0.8,marker='o',markerfacecolor='w',ms=marker_size)
for i, strain in enumerate(strains):
    for j, iptg in enumerate(iptgs):
        ax[j,i].set_title(f'{strain}: {iptg}$\mu$M IPTG',fontsize=14)
        for k, sal in enumerate(sals):
            ax[j,i].plot(df_gfp_avg.loc[(df_gfp_avg.strain==strain) &\
                      (df_gfp_avg.chlor=='+') &\
                      (df_gfp_avg.iptg==iptg) &\
                      (df_gfp_avg.las==las) &\
                      (df_gfp_avg.sal==sal) &\
                      (df_gfp_avg.cole1==cole1),'plate'].values+offsets[2*k+1],
                        df_gfp_avg.loc[(df_gfp_avg.strain==strain) &\
                      (df_gfp_avg.chlor=='+') &\
                      (df_gfp_avg.iptg==iptg) &\
                      (df_gfp_avg.las==las) &\
                      (df_gfp_avg.sal==sal) &\
                      (df_gfp_avg.cole1==cole1),'measurement'],
                             linestyle='-',linewidth=lw,color=chlor_colors[k],alpha=0.8,marker='o',ms=marker_size)
            ax[j,i].set_xticks(np.arange(1,16,2))

legend_list = []
for k, sal in enumerate(sals):
    legend_list.append(f'{sal} $\mu$M -chlor')
for k, sal in enumerate(sals):
    legend_list.append(f'{sal} $\mu$M +chlor')
ax[0,0].legend(legend_list,loc='best',fontsize=10)

strains = np.array(['naive 1x','naive 2x'])
colors=['lightgray','darkgray']
colors=['gray','black']
mfc=['white','black']
for i, strain in enumerate(strains):
    for j, iptg in enumerate(iptgs):
        ax[j,2].set_title(f'Naive: {iptg}$\mu$M IPTG',fontsize=14)
        ax[j,2].plot(df_gfp_avg.loc[(df_gfp_avg.strain==strain) &\
                          (df_gfp_avg.iptg==iptg) &\
                      (df_gfp_avg.cole1==cole1),'plate']+offsets[i+3],
                            df_gfp_avg.loc[(df_gfp_avg.strain==strain) &\
                          (df_gfp_avg.iptg==iptg) &\
                      (df_gfp_avg.cole1==cole1),'measurement'],
                          linewidth=lw,color=naive_colors[i],alpha=0.8,marker='o',ms=marker_size,
                    markerfacecolor=mfc[i])
        ax[j,2].set_xticks(np.arange(1,16,2))
#             ax[j,i].set_ylim(0,5e4)
ax[0,2].legend(['naive 1x','naive 2x'],fontsize=10,loc='best')     
ax[1,2].legend(['naive 1x','naive 2x'],fontsize=10,loc='best')     

ax[0,0].set_ylabel('cumulative sfGFP (AU)',fontsize=14)
ax[1,0].set_ylabel('cumulative sfGFP (AU)',fontsize=14)
ax[1,0].set_xlabel('plate generation',fontsize=14)
ax[1,1].set_xlabel('plate generation',fontsize=14)
ax[1,2].set_xlabel('plate generation',fontsize=14)
ax[0,0].set_yscale('log')
# plt.savefig(f'./20211025_{cole1}_endpoint_timecourse_20220521.pdf') #uncomment to save figure


In [34]:
# generate palplot for legend
sns.palplot(chlor_colors)
plt.savefig('200220521_diverging_plaplot_fig2.pdf')

Below will generate the bargraphs showing total endpoint GFP production after 16 plate generations shown in panel D of Figure 2, Figure 3, and Supplementary Figure 28. The y axis in the plots is the same for each of these, and Figure 3 was generated by combining the bar graphs with Affinity Designer. SD error bars included, and all 8 replicates plotted.

In [34]:
# Make bar graph for Figure S28D (and portion of Figure 3F)
def change_width(ax, new_value):
    for patch in ax.patches:
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)
    return
        
chlor_colors = ['#4575b4','#abd9e9', '#fdae61', '#d73027']
nochlor_colors = ['#4575b4','#abd9e9', '#fdae61', '#d73027']
naive_colors = [(150/255,150/255,150/255),(37/255,37/255,37/255)]
fig, ax = plt.subplots(2,3,sharey=True,sharex='col',figsize=(32,15))  
strains = np.array(['1x diff 2x bxb1','split pir 2x bxb1'])
iptgs = [10,50]
las = 0.01
chlors=['-','+']
cole1 = 'carb'

palettes=[nochlor_colors,chlor_colors]
sns.set_palette(nochlor_colors)
for i, strain in enumerate(strains):
    for j, iptg in enumerate(iptgs):
        ax[j,i].set_title(f'{strain}: {iptg}$\mu$M IPTG',fontsize=14)
        data = df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain)&\
                          (df_gfp_cumsum.iptg==iptg) &\
                          (df_gfp_cumsum.plate==16)&\
                      (df_gfp_cumsum.cole1==cole1),:]
        sns.swarmplot(ax=ax[j,i],x="chlor", y="measurement", hue="sal", data=data,dodge=True,size=4,edgecolor=".2",palette=['black'],alpha=0.8)
        sns.barplot(ax=ax[j,i],x="chlor", y="measurement", hue="sal", data=data,capsize=0.1,errwidth=1.25,alpha=0.8,ci='sd',edgecolor=".2")#,palette=['white','white','white','white']+nochlor_colors)
        ax[j,i].get_legend().set_visible(False)

strains = np.array(['naive 1x','naive 2x'])
sns.set_palette(['gray','black'])
for j, iptg in enumerate(iptgs):
    ax[j,2].set_title(f'Naive: {iptg}$\mu$M IPTG',fontsize=14)
    data = df_gfp_cumsum.loc[((df_gfp_cumsum.strain=='naive 1x')|(df_gfp_cumsum.strain=='naive 2x'))&\
                             (df_gfp_cumsum.plate==16)&\
                             (df_gfp_cumsum.iptg==iptg)&\
                             (df_gfp_cumsum.cole1==cole1),:]


    
    
    data.sort_values(by='strain',inplace=True)
    sns.swarmplot(ax=ax[j,2],x="strain", y="measurement", hue="strain", data=data,dodge=True,size=4,edgecolor=".2",palette=['black'],alpha=0.8)
    sns.barplot(ax=ax[j,2],x="strain", y="measurement", hue="strain", data=data,capsize=0.1,errwidth=1.25,alpha=0.8,ci='sd',edgecolor=".2")
    ax[j,2].get_legend().set_visible(False)
    ax[j,2].set_xlabel("strain", fontsize = 14)
change_width(ax[0,2], 0.2)
change_width(ax[1,2], 0.2)

carb_ylim = ax[0,0].get_ylim() # get y-axis to use for kanR plot below
# plt.savefig(f'./20211025_{cole1}_endpoint_20220521.pdf') # uncomment to save plot

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/seaborn/categorical.py:1296: UserWarning: 12.5% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# Make bar graph for Figure 2D (and portion of Figure 3F)
def change_width(ax, new_value):
    for patch in ax.patches:
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)
    return
        
chlor_colors = ['#4575b4','#abd9e9', '#fdae61', '#d73027']
nochlor_colors = ['#4575b4','#abd9e9', '#fdae61', '#d73027']
naive_colors = [(150/255,150/255,150/255),(37/255,37/255,37/255)]
fig, ax = plt.subplots(2,3,sharey=True,sharex='col',figsize=(32,15))  
strains = np.array(['1x diff 2x bxb1','split pir 2x bxb1'])
iptgs = [10,50]
las = 0.01
chlors=['-','+']
cole1 = 'kan'

palettes=[nochlor_colors,chlor_colors]
sns.set_palette(nochlor_colors)
for i, strain in enumerate(strains):
    for j, iptg in enumerate(iptgs):
        ax[j,i].set_title(f'{strain}: {iptg}$\mu$M IPTG',fontsize=14)
        data = df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain)&\
                          (df_gfp_cumsum.iptg==iptg) &\
                          (df_gfp_cumsum.plate==16)&\
                      (df_gfp_cumsum.cole1==cole1),:]
        sns.swarmplot(ax=ax[j,i],x="chlor", y="measurement", hue="sal", data=data,dodge=True,size=4,edgecolor=".2",palette=['black'],alpha=0.8)
        sns.barplot(ax=ax[j,i],x="chlor", y="measurement", hue="sal", data=data,capsize=0.1,errwidth=1.25,alpha=0.8,ci='sd',edgecolor=".2")#,palette=['white','white','white','white']+nochlor_colors)
        ax[j,i].get_legend().set_visible(False)

strains = np.array(['naive 1x','naive 2x'])
sns.set_palette(['gray','black'])
for j, iptg in enumerate(iptgs):
    ax[j,2].set_title(f'Naive: {iptg}$\mu$M IPTG',fontsize=14)
    data = df_gfp_cumsum.loc[((df_gfp_cumsum.strain=='naive 1x')|(df_gfp_cumsum.strain=='naive 2x'))&\
                             (df_gfp_cumsum.plate==16)&\
                             (df_gfp_cumsum.iptg==iptg)&\
                             (df_gfp_cumsum.cole1==cole1),:]


    
    
    data.sort_values(by='strain',inplace=True)
    sns.swarmplot(ax=ax[j,2],x="strain", y="measurement", hue="strain", data=data,dodge=True,size=4,edgecolor=".2",palette=['black'],alpha=0.8)
    sns.barplot(ax=ax[j,2],x="strain", y="measurement", hue="strain", data=data,capsize=0.1,errwidth=1.25,alpha=0.8,ci='sd',edgecolor=".2")
    ax[j,2].get_legend().set_visible(False)
    ax[j,2].set_xlabel("strain", fontsize = 14)
change_width(ax[0,2], 0.2)
change_width(ax[1,2], 0.2)

ax[0,0].set_ylim(carb_ylim) # use ylim from carb plot (AmpR, Figure S28)
# plt.savefig(f'./20211025_{cole1}_endpoint_20220521.pdf') # uncomment to save plot

In [6]:
df_gfp_cumsum_kan = df_gfp_cumsum.loc[df_gfp_cumsum.cole1=='kan',:]
df_gfp_cumsum_carb = df_gfp_cumsum.loc[df_gfp_cumsum.cole1=='carb',:]
df_gfp_cumsum_sorted = pd.concat([df_gfp_cumsum_kan,df_gfp_cumsum_carb])

Below will generate timecourse plots for naive 1x/2x shown in Supplementary Figure 13.

In [43]:
sns.set_palette('tab10')
fig, ax = plt.subplots(3,4,sharey='row',sharex=True,figsize=(32,15))  

las = 0.01
sals = [10,15,20,30]
strain = '1x diff 2x bxb1'
# strain = 'split pir 2x bxb1'
reps = [1,2,3,4,5,6,7,8]
cole1='carb'
cole1='kan'
iptgs = [10,50]
strains= ['naive 1x','naive 2x']
fig, ax = plt.subplots(3,8,sharey='row',sharex=True,figsize=(32,6))  
for c, cole1 in enumerate(['carb','kan']):
#     fig, ax = plt.subplots(3,4,sharey='row',sharex=True,figsize=(16,15)) 
    for i, strain in enumerate(strains):
        for j, iptg in enumerate(iptgs):
            for k, rep in enumerate(reps):
                ax[0,2*i+j+4*c].set_title(f'{strain},{cole1}:{iptg}$\mu$M IPTG',fontsize=14)
                ax[0,2*i+j+4*c].plot(df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.iptg==iptg) &\
                                  (df_gfp_cumsum.rep==rep)&\
                                  (df_gfp_cumsum.cole1==cole1),'plate'],
                                    df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.iptg==iptg) &\
                                  (df_gfp_cumsum.rep==rep)&\
                                  (df_gfp_cumsum.cole1==cole1),'measurement'],'-',color=sns.color_palette()[k],alpha=0.8)
                
                ax[1,2*i+j+4*c].plot(df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.iptg==iptg) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'plate'],
                            df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.iptg==iptg) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'measurement'],'-',color=sns.color_palette()[k],alpha=0.8)
                ax[2,2*i+j+4*c].plot(df_od.loc[(df_od.strain==strain) &\
                          (df_od.iptg==iptg) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'plate'],
                            df_od.loc[(df_od.strain==strain) &\
                          (df_od.iptg==iptg) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'measurement'],'-',color=sns.color_palette()[k],alpha=0.8)

        ax[1,0].set_ylim(0,200)
        ax[0,0].set_ylabel('cumulative sfGFP (a.u.)',fontsize=14)
        ax[1,0].set_ylabel('mScarlet (a.u.)',fontsize=14)
        ax[2,0].set_ylabel('OD700',fontsize=14)
    for i in range(8):
        ax[2,i].set_xlabel('plate generation',fontsize=14)

# plt.savefig(f'./20211025_naive1x2x_carb_kan_gfp_rfp_OD.png') #uncomment to save figures
# plt.savefig(f'./20211025_naive1x2x_carb_kan_gfp_rfp_OD.pdf') #uncomment to save figures

Below will generate timecourse plots for differentiation/terminal differentiation shown in Supplementary Figures 14-15.

In [41]:
sns.set_palette('tab10')
iptgs = [10,50]
las = 0.01
sals = [10,15,20,30]
strain = '1x diff 2x bxb1'
# strain = 'split pir 2x bxb1'
reps = [1,2,3,4,5,6,7,8]
cole1='carb'
cole1='kan'
for strain in ['1x diff 2x bxb1','split pir 2x bxb1']:
    for cole1 in ['carb','kan']:
        fig, ax = plt.subplots(3,8,sharey='row',sharex=True,figsize=(32,6))  
        for i, sal in enumerate(sals):
            for k, rep in enumerate(reps):
                ax[0,i].plot(df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='-') &\
                                  (df_gfp_cumsum.iptg==iptgs[0]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                                  (df_gfp_cumsum.cole1==cole1),'plate'],
                                    df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='-') &\
                                  (df_gfp_cumsum.iptg==iptgs[0]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                                  (df_gfp_cumsum.cole1==cole1),'measurement'],'--',color=sns.color_palette()[k],alpha=0.8)

                ax[1,i].plot(df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='-') &\
                          (df_rfp.iptg==iptgs[0]) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'plate'],
                            df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='-') &\
                          (df_rfp.iptg==iptgs[0]) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'measurement'],'--',color=sns.color_palette()[k],alpha=0.8)
                ax[2,i].plot(df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='-') &\
                          (df_od.iptg==iptgs[0]) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'plate'],
                            df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='-') &\
                          (df_od.iptg==iptgs[0]) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'measurement'],'--',color=sns.color_palette()[k],alpha=0.8)

                ax[0,i].plot(df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='+') &\
                                  (df_gfp_cumsum.las==las) &\
                                  (df_gfp_cumsum.iptg==iptgs[0]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                              (df_gfp_cumsum.cole1==cole1),'plate'].values,
                                    df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='+') &\
                                  (df_gfp_cumsum.las==las) &\
                                  (df_gfp_cumsum.iptg==iptgs[0]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                              (df_gfp_cumsum.cole1==cole1),'measurement'].values,'-',color=sns.color_palette()[k],alpha=0.8)


                ax[1,i].plot(df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='+') &\
                          (df_rfp.iptg==iptgs[0]) &\
                          (df_rfp.las==las) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'plate'].values,
                            df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='+') &\
                          (df_rfp.iptg==iptgs[0]) &\
                          (df_rfp.las==las) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'measurement'].values,'-',color=sns.color_palette()[k],alpha=0.8)
                ax[2,i].plot(df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='+') &\
                          (df_od.iptg==iptgs[0]) &\
                          (df_od.las==las) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'plate'].values,
                            df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='+') &\
                          (df_od.iptg==iptgs[0]) &\
                          (df_od.las==las) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'measurement'].values,'-',color=sns.color_palette()[k],alpha=0.8)
                ax[0,i].set_title(f'{iptgs[0]}$\mu$M IPTG,{sal}$\mu$M Sal',fontsize=8)
        for i, sal in enumerate(sals):
            for k, rep in enumerate(reps):
                ax[0,i+4].plot(df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='-') &\
                                  (df_gfp_cumsum.iptg==iptgs[1]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                                  (df_gfp_cumsum.cole1==cole1),'plate'],
                                    df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='-') &\
                                  (df_gfp_cumsum.iptg==iptgs[1]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                                  (df_gfp_cumsum.cole1==cole1),'measurement'],'--',color=sns.color_palette()[k],alpha=0.8)

                ax[1,i+4].plot(df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='-') &\
                          (df_rfp.iptg==iptgs[1]) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'plate'],
                            df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='-') &\
                          (df_rfp.iptg==iptgs[1]) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'measurement'],'--',color=sns.color_palette()[k],alpha=0.8)
                ax[2,i+4].plot(df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='-') &\
                          (df_od.iptg==iptgs[1]) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'plate'],
                            df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='-') &\
                          (df_od.iptg==iptgs[1]) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'measurement'],'--',color=sns.color_palette()[k],alpha=0.8)

                ax[0,i+4].plot(df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='+') &\
                                  (df_gfp_cumsum.las==las) &\
                                  (df_gfp_cumsum.iptg==iptgs[1]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                              (df_gfp_cumsum.cole1==cole1),'plate'].values,
                                    df_gfp_cumsum.loc[(df_gfp_cumsum.strain==strain) &\
                                  (df_gfp_cumsum.chlor=='+') &\
                                  (df_gfp_cumsum.las==las) &\
                                  (df_gfp_cumsum.iptg==iptgs[1]) &\
                                  (df_gfp_cumsum.sal==sal) &\
                                  (df_gfp_cumsum.rep==rep)&\
                              (df_gfp_cumsum.cole1==cole1),'measurement'].values,'-',color=sns.color_palette()[k],alpha=0.8)

                ax[1,i+4].plot(df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='+') &\
                          (df_rfp.iptg==iptgs[1]) &\
                          (df_rfp.las==las) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'plate'].values,
                            df_rfp.loc[(df_rfp.strain==strain) &\
                          (df_rfp.chlor=='+') &\
                          (df_rfp.iptg==iptgs[1]) &\
                          (df_rfp.las==las) &\
                          (df_rfp.sal==sal) &\
                          (df_rfp.rep==rep)&\
                      (df_rfp.cole1==cole1),'measurement'].values,'-',color=sns.color_palette()[k],alpha=0.8)

                ax[2,i+4].plot(df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='+') &\
                          (df_od.iptg==iptgs[1]) &\
                          (df_od.las==las) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'plate'].values,
                            df_od.loc[(df_od.strain==strain) &\
                          (df_od.chlor=='+') &\
                          (df_od.iptg==iptgs[1]) &\
                          (df_od.las==las) &\
                          (df_od.sal==sal) &\
                          (df_od.rep==rep)&\
                      (df_od.cole1==cole1),'measurement'].values,'-',color=sns.color_palette()[k],alpha=0.8)
                ax[0,i+4].set_title(f'{iptgs[1]}$\mu$M IPTG,{sal}$\mu$M Sal',fontsize=8)     
                ax[2,0].set_ylim(0,1.8)
        ax[0,0].set_ylabel('cumulative sfGFP (a.u.)',fontsize=14)
        ax[1,0].set_ylabel('mScarlet (a.u.)',fontsize=14)
        ax[2,0].set_ylabel('OD700',fontsize=14)
        for i in range(8):
            ax[2,i].set_xlabel('plate generation',fontsize=14)

#         plt.savefig(f'./20211025_{strain}_{cole1}_gfp_rfp_OD.png') #uncomment to save figures
#         plt.savefig(f'./20211025_{strain}_{cole1}_gfp_rfp_OD.pdf') #uncomment to save figures